In [39]:
import requests 
import json
import time 
import tqdm
import pandas as pd

df = pd.read_csv('Boliga Scraped.csv')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 500
df.sort_values(by = ['municipality'], axis = 0 )

,Unnamed: 0,Unnamed: 0.1,agentDisplayName,agentRegId,area,basementSize,buildYear,city,createdDate,daysForSale,domainId,energyClass,exp,floor,groupKey,guid,id,images,inWatchlist,isActive,isForeclosure,isPremiumAgent,latitude,longitude,lotSize,municipality,net,openHouse,price,priceChangePercentTotal,propertyType,rooms,selfsale,size,squaremeterPrice,street,views,zipCode,addr_full,coordinate
36936,36936,36,NaN,32,0,0,2020,Ringkøbing,2019-05-25T03:14:17.827Z,90,9,-,4038,NaN,1.0,NaN,1560766,"[{'id': 1560766, 'date': '2019-08-23T15:05:11....",False,True,False,False,0.00000,0.00000,300,0,0,NaN,1368000,0,4,3.0,False,48,28500.0,Active 4,1,6950,Active 4 Ringkøbing,"0.0, 0.0"
15884,15884,34,Holbæk - Henrik Larsen A/S,885,0,0,0,Vipperød,2019-02-28T03:25:11.560Z,176,6,-,1688,NaN,1.0,NaN,1527579,"[{'id': 1527579, 'date': '2019-08-23T14:54:30....",False,True,False,True,0.00000,0.00000,1000,0,0,NaN,695000,0,7,0.0,False,0,0.0,Frugtparken Syd 40,0,4390,Frugtparken Syd 40 Vipperød,"0.0, 0.0"
15877,15877,27,Holbæk - Henrik Larsen A/S,885,0,0,0,Vipperød,2019-02-28T03:25:11.697Z,176,6,-,1479,NaN,1.0,NaN,1527580,"[{'id': 1527580, 'date': '2019-08-23T14:54:30....",False,True,False,True,0.00000,0.00000,1000,0,0,NaN,595000,0,7,0.0,False,0,0.0,Frugtparken Syd 33,0,4390,Frugtparken Syd 33 Vipperød,"0.0, 0.0"
15874,15874,24,Holbæk - Henrik Larsen A/S,885,0,0,0,Vipperød,2019-02-28T03:25:11.940Z,176,6,-,1379,NaN,1.0,NaN,1527583,"[{'id': 1527583, 'date': '2019-08-23T14:54:30....",False,True,False,True,0.00000,0.00000,1000,0,0,NaN,550000,0,7,0.0,False,0,0.0,Frugtparken Syd 23,1,4390,Frugtparken Syd 23 Vipperød,"0.0, 0.0"
15811,15811,11,Holbæk - Henrik Larsen A/S,885,0,60,1939,Tølløse,2019-05-23T03:07:36.000Z,92,6,G,4284,NaN,1.0,9D11211F-1CD5-4E12-95F3-CA261D9265A4,1559702,"[{'id': 1559702, 'date': '2019-08-23T14:54:29....",False,True,False,True,55.64617,11.71766,694332,0,0,NaN,13995000,0,6,8.0,False,220,63613.0,Gl. Skovvej 64,5,4340,Gl. Skovvej 64 Tølløse,"55.64617, 11.71766"
15807,15807,7,Holbæk - Henrik Larsen A/S,885,0,60,1939,Tølløse,2019-05-23T03:07:36.000Z,92,6,G,4466,NaN,1.0,9D11211F-1CD5-4E12-95F3-CA261D9265A4,1559709,"[{'id': 1559709, 'date': '2019-08-23T14:54:29....",False,True,False,True,55.64617,11.71766,122355,0,0,NaN,4495000,0,6,8.0,False,220,20431.0,Gl. Skovvej 64,1,4340,Gl. Skovvej 64 Tølløse,"55.64617, 11.71766"
15797,15797,47,Holbæk - Henrik Larsen A/S,885,0,60,1939,Tølløse,2019-05-23T03:08:51.000Z,92,6,G,4354,NaN,NaN,9D11211F-1CD5-4E12-95F3-CA261D9265A4,1559807,"[{'id': 1559807, 'date': '2019-08-23T14:54:27....",False,True,False,True,55.64617,11.71766,19900,0,0,NaN,2995000,0,1,8.0,False,220,13613.0,Gl. Skovvej 64,5,4340,Gl. Skovvej 64 Tølløse,"55.64617, 11.71766"
15578,15578,28,Holbæk - Henrik Larsen A/S,885,0,0,1905,Holbæk,2019-02-15T03:55:41.000Z,189,6,B,3348,NaN,NaN,C36F14CD-6CD9-4B09-A24C-B9866DCAAE03,1523648,"[{'id': 1523648, 'date': '2019-08-23T14:54:21....",False,True,False,True,55.73051,11.63263,15000,0,0,NaN,8800000,-5,6,5.0,False,231,38095.0,Gl. Mårsøvej 5,4,4300,Gl. Mårsøvej 5 Holbæk,"55.730509999999995, 11.63263"
15168,15168,18,NaN,570,0,0,0,Dianalund,2019-08-17T03:29:32.423Z,6,9,-,443,NaN,NaN,NaN,1586599,NaN,False,True,False,False,0.00000,0.00000,65936,0,0,NaN,1200000,0,7,0.0,False,0,0.0,Del af Holbergsvej 51 og Holbækvej 1,7,4293,Del af Holbergsvej 51 og Holbækvej 1 Dianalund,"0.0, 0.0"
13842,13842,42,NaN,0,0,0,0,Slagelse,2018-12-12T00:41:08.263Z,254,833,-,2681,NaN,NaN,NaN,1508619,"[{'id': 1508619, 'date': '2019-08-23T14:53:24....",False,True,False,False,0.00000,0.00000,780,0,3419,NaN,849000,0,7,0.0,False,0,0.0,Høje Bjergby 1,0,4200,Høje Bjergby 1 Slagelse,"0.0, 0.0"


In [55]:
#Oprydning
df.drop(df.loc[df['municipality']==0].index, inplace = True)
df.drop(df.loc[df['price']<=100].index, inplace = True)
df.drop(df.loc[df['latitude']==0].index, inplace = True)
df.drop(df.loc[df['longitude']==0].index, inplace = True)

#Unique municipality df
kommune = df['municipality'].unique()
df_kom = pd.DataFrame(kommune, columns = ['Kommunenr'])

In [61]:
import scraping_class
logfile = 'Boligscrape_log.csv'
connector = scraping_class.Connector(logfile)

import time
def kommune_func(kommune_kode):
    url = lambda q: 'https://dawa.aws.dk/kommuner/{}'.format(q)
    resp, call_id = connector.get(url(kommune_kode), 'Kommunedata')
    time.sleep(0.5)
    
    if not resp.status_code == 200:
        return ''
    resp = resp.json()
    kommune = resp['navn']
    
    return kommune

In [69]:
import scraping_class
logfile = 'Boligscrape_log.csv'
connector = scraping_class.Connector(logfile)

import time
def region_func(kommune_kode):
    url = lambda q: 'https://dawa.aws.dk/kommuner/{}'.format(q)
    resp, call_id = connector.get(url(kommune_kode), 'Kommunedata')
    time.sleep(0.5)
    
    if not resp.status_code == 200:
        return ''
    resp = resp.json()
    region = resp['region']['navn']
    
    return region

In [63]:
df_kom['Kommune'] = df_kom.Kommunenr.apply(kommune_func)

In [70]:
df_kom['Region'] = df_kom.Kommunenr.apply(region_func)

In [72]:
df = df.merge(df_kom, how='left', left_on=['municipality'],
                           right_on=['Kommunenr'], validate = 'm:1')

In [83]:
df = df.drop(columns = ['Unnamed: 0'], axis = 1)
df = df.drop(columns = ['Unnamed: 0.1'], axis = 1)
df = df.drop(columns = ['Kommunenr'], axis = 1)

In [139]:
# Convert Prop_type (number) to descriptive labels
prop_type = {'Prop_Type': {'Villa': 1, 'Rækkehus': 2, 'Ejerlejlighed': 3,
            'Fritidshus': 4, 'Andelsbolig': 5, 'Landejendom': 6,
            'Helårsgrund': 7, 'Fritidsgrund': 8, 'Villalejlighed': 9}}

prop_type_df = pd.DataFrame(prop_type).reset_index()

df = df.merge(prop_type_df, how='left', left_on=['propertyType'],
                           right_on=['Prop_Type'], validate = 'm:1')

df = df.drop(columns = ['Prop_Type'], axis = 1)
df.rename(columns={'index':'Type'}, inplace=True)
#Drop missing in Type - those are tvangsaktioner
df = df[df.Type.isnull() != True]

In [23]:
df.to_csv('Boliga - Behanling1 - Kom, Reg, Type.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
